从pickle中获取删除了空行空列的数据

In [ ]:
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms.community import greedy_modularity_communities

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

with open('../data/data_remove_empty.pkl', 'rb') as file:
    data = pickle.load(file)

data

数据中有特殊符号，它们在医学中的含义分别是：
- <X: 表示低于实验室检测下限
- \>X: 表示高于实验室检测上限
- ?X: 表示检测结果存疑

要把数据喂给数值模型，必须把它们转换为数值类型，考虑到这些数据的占比非常小，对它们分别的处理方式是：
- <X: 用X/$\sqrt{2}$替代
- \>X: 用X*1.1替代
- ?X: 移除?，暂时接受不可靠的数据

In [ ]:
for col in data.iloc[:, 8:].columns:
    if data[col].dtype == 'object':
        data[col] = data[col].astype(str).str.replace(r'<(\d+\.?\d*)', lambda m: str(float(m.group(1)) / np.sqrt(2)),
                                                      regex=True)
        data[col] = data[col].astype(str).str.replace(r'>(\d+\.?\d*)', lambda m: str(float(m.group(1)) * 1.1),
                                                      regex=True)
        data[col] = data[col].astype(str).str.replace(r'\?(\d+\.?\d*)', lambda m: m.group(1), regex=True)
        data[col] = pd.to_numeric(data[col], errors='coerce')
data

有些列列名相同，比如甘油三酯 甘油三酯1，其实是实验室中同一样本重复测量的结果，可以聚合成平均值。
葡萄糖1 2 3可能是不同时间测的同一个指标葡萄糖，葡萄糖实际指空腹血糖，它可以是葡萄糖1 2 3的任意一个，意图可能是多次测量避免血糖的波动，因此可以将葡萄糖1 2 3删去。甲状旁腺激素（pg/ml）1其实是甲状旁腺激素同一个值的不同计量方式，我们可以把它直接删除。

In [ ]:
data_rem_dup = data.copy()

merge_map = {
    '游离三碘甲状腺原氨酸': ['游离三碘甲状腺原氨酸', '游离三碘甲状腺原氨酸1'],
    '碱性磷酸酶': ['碱性磷酸酶', '碱性磷酸酶1'],
    '甲状旁腺激素': ['甲状旁腺激素', '甲状旁腺激素1'],
    '降钙素': ['降钙素', '降钙素1', '降钙素2'],
    '总胆红素': ['总胆红素', '总胆红素1'],
    '维生素B12': ['维生素B12', '维生素B12 1'],
    '白介素1β': ['白介素1β', '白介素1β 1'],
    '白介素2': ['白介素2', '白介素2 1'],
    '白介素4': ['白介素4', '白介素4 1'],
    '白介素5': ['白介素5', '白介素5 1'],
    '白介素6': ['白介素6', '白介素6 1'],
    '白介素8': ['白介素8', '白介素8 1'],
    '白介素10': ['白介素10', '白介素10 1'],
    '白介素17A': ['白介素17A', '白介素17A 1'],
    '肿瘤坏死因子α': ['肿瘤坏死因子α', '肿瘤坏死因子α 1'],
    '干扰素γ': ['干扰素γ', '干扰素γ 1'],
    '25-羟基维生素D': ['25-羟基维生素D', '25-羟基维生素D1', '25-羟基维生素D2'],
}

# 执行合并
for target_col, cols in merge_map.items():
    existing_cols = [col for col in cols if col in data_rem_dup.columns]
    if len(existing_cols) > 1:
        data_rem_dup[target_col] = data_rem_dup[existing_cols].mean(axis=1, skipna=True)
        drop_cols = [col for col in existing_cols if col != target_col]
        data_rem_dup.drop(columns=drop_cols, inplace=True)

# 删除多余列
for col in ['葡萄糖1', '葡萄糖2', '葡萄糖3', '甲状旁腺激素（pg/ml）1']:
    if col in data_rem_dup.columns:
        data_rem_dup.drop(columns=col, inplace=True)

# 将带1后缀的列改名
data_rem_dup.rename(columns=lambda col: col.rstrip('1') if col in ['甘油三酯1', 'C肽1'] else col, inplace=True)

# 保留第8列以后的检测指标
value_data = data_rem_dup.iloc[:, 8:]

data_rem_dup.to_pickle('../data/data_remove_dup.pkl')

data_rem_dup

这份数据的稀疏性是有道理的，因为医生会根据初步诊断决定要给病人检测什么指标，特征是否缺失，以及特征之间缺失的关联也具有分析的价值。我们可以计算特征之间的共现占比矩阵，并画共现热图来分析特征之间的共现关联。

我们可以筛选贡献占比大于0.8且共现次数大于100的特征对，来分析它们之间的关系，它们的共现关系是比较可靠的。利用图论算法，我们可以得到这些特征对的连通块，连通块中的特征对之间是有共现关系的。以防万一，我们还要计算连通子图的密度，避免被稀疏联通或甚至是链式联通误导。

In [ ]:
# 计算共现占比矩阵
co_occurrence = value_data.notnull().astype(int).T.dot(value_data.notnull().astype(int))
co_occurrence_ratio = co_occurrence.div(value_data.notnull().sum(axis=0), axis=1)

# 画热图
plt.figure(figsize=(18, 14))
sns.heatmap(co_occurrence_ratio, cmap='coolwarm', annot=False, fmt="d", square=True,
            mask=np.triu(np.ones(co_occurrence_ratio.shape, dtype=bool)), cbar_kws={"shrink": .8})
plt.title("临床检测指标共现热图", fontsize=16)
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# 筛选共现占比大于0.8且共现次数大于100的对
co_occurrence_pairs = []
for i in range(co_occurrence.shape[0]):
    for j in range(i + 1, co_occurrence.shape[1]):
        if co_occurrence.iloc[i, j] > 100 and co_occurrence_ratio.iloc[i, j] > 0.8:
            co_occurrence_pairs.append((co_occurrence.index[i], co_occurrence.columns[j]))

corr_graph = nx.Graph()
corr_graph.add_edges_from(co_occurrence_pairs)
corr_connected_components = list(nx.connected_components(corr_graph))

# 计算每个连通块的子图密度
component_densities = [
    (component, nx.density(corr_graph.subgraph(component)))
    for component in corr_connected_components
]

component_densities

第一个连通块有点稀疏，我们用社区检测算法来进一步将其划分为无重叠的子图。

In [ ]:
sparse_component = corr_connected_components[0]

# 使用社区检测算法
communities = list(greedy_modularity_communities(corr_graph.subgraph(sparse_component)))

# 附加每个社区的子图密度
community_densities = [
    (community, nx.density(corr_graph.subgraph(community)))
    for community in communities
]

community_densities

到此为止，整个数据集的数值特征已经被分为以下部分，其中每一个似乎都是针对某一类疾病的相关指标，具有一定临床意义。

1. 餐后耐糖/胰岛素
2. 甲状腺功能
3. 骨形成/TRAb，TRAb是骨代谢检查的病因筛查拓展项
4. 血钙-骨代谢调节轴的核心三激素
5. 空腹胰岛素释放
6. 生殖／性激素
7. 免疫／炎症因子
8. 代谢综合征
9. 肾功能
10. 肝胆功能

还剩下一些没有被分组的特征，它们可能和其它特征关系没有那么密切，或者是相对独立的特征。我们可以计算它们与其他特征的共现频次，手动处理这些特征。

In [ ]:
groups = [['C肽（餐后2小时）', '胰岛素（餐后2小时）', '葡萄糖(餐后2小时)'],
          ['促甲状腺素', '反三碘甲状腺原氨酸', '总三碘甲状腺原氨酸', '总四碘甲状腺原氨酸', '游离三碘甲状腺原氨酸',
           '游离甲状腺素', '甲状腺球蛋白', '甲状腺过氧化物酶抗体'],
          ['β-胶原特殊序列', '促甲状腺素受体抗体', '总I型胶原氨基端延长肽', '骨钙素(N-MID)'],
          ['25-羟基维生素D', '甲状旁腺激素', '降钙素'],
          ['C肽', '胰岛素'],
          ['促卵泡成熟素', '促黄体生成素', '叶酸', '孕酮', '泌乳素', '睾酮', '硫酸去氢表雄酮', '雌二醇'],
          ['干扰素γ', '白介素10', '白介素17A', '白介素1β', '白介素2', '白介素2受体', '白介素4', '白介素5', '白介素6',
           '白介素8', '肿瘤坏死因子α'],
          ['低密度脂蛋白', '总胆固醇', '甘油三酯', '磷', '糖化白蛋白', '葡萄糖', '镁', '高密度脂蛋白'],
          ['尿素', '尿酸', '氯', '肌酐', '钙', '钠', '钾'],
          ['γ-谷氨酰转肽酶', '天门冬氨酸转氨酶', '总胆红素', '直接胆红素', '碱性磷酸酶']]

all_features = set(value_data.columns)

grouped_features = set(feature for group in groups for feature in group)

# 找出未涉及的特征
ungrouped_features = all_features - grouped_features

# 计算每个 ungrouped_feature 与其他特征的共同出现频次
ungrouped_feature_co_occurrence = {}

for feature in ungrouped_features:
    co_occurrence_counts = value_data.notnull().astype(int).T.dot(value_data[feature].notnull().astype(int))
    co_occurrence_counts = co_occurrence_counts.sort_values(ascending=False)
    ungrouped_feature_co_occurrence[feature] = co_occurrence_counts

ungrouped_feature_co_occurrence_df = pd.DataFrame.from_dict(
    {feature: counts for feature, counts in ungrouped_feature_co_occurrence.items()},
    orient='index'
).fillna(0)

ungrouped_feature_co_occurrence_df = ungrouped_feature_co_occurrence_df.stack().reset_index()
ungrouped_feature_co_occurrence_df.columns = ['Feature1', 'Feature2', 'Frequency']
ungrouped_feature_co_occurrence_df = ungrouped_feature_co_occurrence_df.sort_values(by=['Feature1', 'Frequency'],
                                                                                    ascending=False)

ungrouped_feature_co_occurrence_df

我们发现骨源碱性磷酸酶和糖化血红蛋白偏向于独立，和其它指标共现的频次都相当；维生素B12和性激素共现最多，我们认为它可以纳入性激素面板；尿钙、尿肌酐、24小时尿磷三者显然组成了尿常规检查面板，因为样本数太少它在第一次分组中被剔除了，我们可以手动加上它们。结果如下：


| 检测分类    | 检测项目                                                                                              | 检测内容标题     |
|---------|---------------------------------------------------------------------------------------------------|------------|
| 血糖血脂相关  | ('C肽（餐后2小时）', '胰岛素（餐后2小时）', '葡萄糖(餐后2小时)')                                                         | 餐后血糖调节     |
|         | ('C肽', '胰岛素')                                                                                     | 空腹胰岛素分泌    |
|         | ('糖化血红蛋白',)                                                                                       | 糖尿病监测      |
|         | ('低密度脂蛋白', '总胆固醇', '甘油三酯', '磷', '糖化白蛋白', '葡萄糖', '镁', '高密度脂蛋白')                                    | 血脂和基础代谢评估  |
| 肾功能相关   | ('尿素', '尿酸', '氯', '肌酐', '钙', '钠', '钾')                                                            | 肾功能及电解质评估  |
|         | ('尿肌酐', '尿钙', '24小时尿磷')                                                                           | 肾功能及钙磷代谢评估 |
| 骨代谢相关   | ('β-胶原特殊序列', '促甲状腺素受体抗体', '总I型胶原氨基端延长肽', '骨钙素(N-MID)')                                            | 骨形成与甲状腺排查  |
|         | ('25-羟基维生素D', '甲状旁腺激素', '降钙素')                                                                    | 血钙-骨代谢调节评估 |
|         | ('骨源碱性磷酸酶',)                                                                                      | 骨形成标志物     |
| 产科相关    | ('促卵泡成熟素', '促黄体生成素', '叶酸', '孕酮', '泌乳素', '睾酮', '硫酸去氢表雄酮', '雌二醇', '维生素B12')                         | 性激素水平与营养评估 |
| 甲状腺相关   | ('促甲状腺素', '反三碘甲状腺原氨酸', '总三碘甲状腺原氨酸', '总四碘甲状腺原氨酸', '游离三碘甲状腺原氨酸', '游离甲状腺素', '甲状腺球蛋白', '甲状腺过氧化物酶抗体')  | 甲状腺功能评估    |
| 免疫/炎症相关 | ('干扰素γ', '白介素10', '白介素17A', '白介素1β', '白介素2', '白介素2受体', '白介素4', '白介素5', '白介素6', '白介素8', '肿瘤坏死因子α') | 炎症因子监测     |
| 肝功能相关   | ('γ-谷氨酰转肽酶', '天门冬氨酸转氨酶', '总胆红素', '直接胆红素', '碱性磷酸酶')                                                | 肝酶及胆红素代谢   |



In [ ]:
groups = [['C肽（餐后2小时）', '胰岛素（餐后2小时）', '葡萄糖(餐后2小时)'],
          ['促甲状腺素', '反三碘甲状腺原氨酸', '总三碘甲状腺原氨酸', '总四碘甲状腺原氨酸', '游离三碘甲状腺原氨酸',
           '游离甲状腺素', '甲状腺球蛋白', '甲状腺过氧化物酶抗体'],
          ['β-胶原特殊序列', '促甲状腺素受体抗体', '总I型胶原氨基端延长肽', '骨钙素(N-MID)'],
          ['25-羟基维生素D', '甲状旁腺激素', '降钙素'],
          ['C肽', '胰岛素'],
          ['促卵泡成熟素', '促黄体生成素', '叶酸', '孕酮', '泌乳素', '睾酮', '硫酸去氢表雄酮', '雌二醇', '维生素B12'],
          ['干扰素γ', '白介素10', '白介素17A', '白介素1β', '白介素2', '白介素2受体', '白介素4', '白介素5', '白介素6',
           '白介素8', '肿瘤坏死因子α'],
          ['低密度脂蛋白', '总胆固醇', '甘油三酯', '磷', '糖化白蛋白', '葡萄糖', '镁', '高密度脂蛋白'],
          ['尿素', '尿酸', '氯', '肌酐', '钙', '钠', '钾'],
          ['γ-谷氨酰转肽酶', '天门冬氨酸转氨酶', '总胆红素', '直接胆红素', '碱性磷酸酶'],
          ['尿肌酐', '尿钙', '24小时尿磷'],
          ['糖化血红蛋白'],
          ['骨源碱性磷酸酶']]

# 初始化组间共现矩阵
group_co_occurrence = pd.DataFrame(0, index=range(len(groups)), columns=range(len(groups)))

# 遍历每个住院号（病人）
for _, group_data in data_rem_dup.groupby('住院号码'):
    # 提取这个病人实际出现过的非空指标
    indicators_present = set()
    for col in group_data.columns:
        if group_data[col].notnull().any():
            indicators_present.add(col)

    # 判断他做了哪些组（70%指标以上出现）
    present_groups = set()
    for i, group in enumerate(groups):
        count_present = sum(1 for item in group if item in indicators_present)
        if count_present / len(group) >= 0.7:
            present_groups.add(i)

    # 更新共现矩阵
    for g1 in present_groups:
        for g2 in present_groups:
            group_co_occurrence.loc[g1, g2] += 1

group_co_occurrence = group_co_occurrence.div(group_co_occurrence.values.diagonal(), axis=1)

# 绘制热力图
plt.figure(figsize=(10, 8))
sns.heatmap(group_co_occurrence, cmap='coolwarm', annot=True, fmt=".2f", square=True, cbar_kws={"shrink": .8})
plt.title("组间共现矩阵热力图", fontsize=16)
plt.xlabel("组索引")
plt.ylabel("组索引")
plt.tight_layout()
plt.show()

group_co_occurrence

通过组间共现热力图，我们看到01234578和11的行比较亮，说明这些组比较通用和基础，经常和其它组一起出现；而其他组的行比较暗，它们相对比较独立，可能用来检测一些特定方向的疾病。

画相关度矩阵热力图来进行特征的相关度分析，尝试找到特征之间的隐藏联系。我们会重点观察相关性在0.6以上的特征对，包括反相关。

原数据十分稀疏，有的特征对相关性甚至算不出来，因为它们从没有一起出现过；有的能算出来，但是因为数据量太少，相关性不可靠。因此我们只考虑非空数据行数大于100的特征对。

In [ ]:
# 计算相关性矩阵
corr = value_data.corr(method='pearson')

# 画热图
plt.figure(figsize=(18, 14))
sns.heatmap(corr, cmap='coolwarm', annot=False, fmt=".2f", square=True,
            mask=corr.isnull() | np.triu(np.ones(corr.shape, dtype=bool)), cbar_kws={"shrink": .8})
plt.title("临床检测指标相关性热图", fontsize=16)
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# 上三角矩阵索引
upper_tri = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))

# 找出绝对值大于0.6的列对
high_corr_pairs = upper_tri.stack().sort_values(ascending=False)
high_corr_pairs = high_corr_pairs[abs(high_corr_pairs) > 0.6]

# 计算相关性来源的非空数据行数
non_null_counts = {
    (col1, col2): value_data[[col1, col2]].dropna().shape[0]
    for col1, col2 in high_corr_pairs.index
}

# 将相关性和非空数据行数一起显示，并筛掉数据来源少于 100 个的
high_corr_pairs = high_corr_pairs.to_frame('相关性')
high_corr_pairs['非空数据行数'] = [
    non_null_counts[(col1, col2)] for col1, col2 in high_corr_pairs.index
]
high_corr_pairs = high_corr_pairs[high_corr_pairs['非空数据行数'] >= 100]

high_corr_pairs

In [ ]:
# 构建图并提取连通块
corr_graph = nx.Graph()
corr_graph.add_edges_from(high_corr_pairs.index)  # 使用相关对作为边
corr_connected_components = list(nx.connected_components(corr_graph))  # 获取连通块

# 输出连通块
corr_connected_components

每一组内的特征之间具有相关性，虽然不是完全相关，比如违反相关性的数据有可能是某些疾病导致的反常，但是这些分组仍然可解释。

1. 免疫和过敏反应
2. 炎症反应
3. 性腺功能
4. 糖代谢
5. 总体和部分的关系，确实应该相关
6. 甲状腺功能
7. 酸碱平衡，肾功能相关
8. 骨代谢
9. 总体和部分的关系
10. 反应肾小球滤过率
11. 共同维持血浆渗透压
12. 胰岛功能

根据上述特征分组，对数据进行进一步拆分，分别存储在不同文件中

In [ ]:

group_map = {
    '餐后血糖调节': ('C肽（餐后2小时）', '胰岛素（餐后2小时）', '葡萄糖(餐后2小时)'),
    '空腹胰岛素分泌': ('C肽', '胰岛素'),
    '糖尿病监测': ('糖化血红蛋白',),
    '血脂和基础代谢评估': ('低密度脂蛋白', '总胆固醇', '甘油三酯', '磷', '糖化白蛋白', '葡萄糖', '镁', '高密度脂蛋白'),
    '肾功能及电解质评估': ('尿素', '尿酸', '氯', '肌酐', '钙', '钠', '钾'),
    '肾功能及钙磷代谢评估': ('尿肌酐', '尿钙', '24小时尿磷'),
    '骨形成与甲状腺排查': ('β-胶原特殊序列', '促甲状腺素受体抗体', '总I型胶原氨基端延长肽', '骨钙素(N-MID)'),
    '血钙-骨代谢调节评估': ('25-羟基维生素D', '甲状旁腺激素', '降钙素'),
    '骨形成标志物': ('骨源碱性磷酸酶',),
    '性激素水平与营养评估': ('促卵泡成熟素', '促黄体生成素', '叶酸', '孕酮', '泌乳素', '睾酮', '硫酸去氢表雄酮', '雌二醇', '维生素B12'),
    '甲状腺功能评估': ('促甲状腺素', '反三碘甲状腺原氨酸', '总三碘甲状腺原氨酸', '总四碘甲状腺原氨酸', 
                  '游离三碘甲状腺原氨酸', '游离甲状腺素', '甲状腺球蛋白', '甲状腺过氧化物酶抗体'),
    '炎症因子监测': ('干扰素γ', '白介素10', '白介素17A', '白介素1β', '白介素2', '白介素2受体', 
               '白介素4', '白介素5', '白介素6', '白介素8', '肿瘤坏死因子α'),
    '肝酶及胆红素代谢': ('γ-谷氨酰转肽酶', '天门冬氨酸转氨酶', '总胆红素', '直接胆红素', '碱性磷酸酶')
}

for group_title, item_tuple in group_map.items():
    cols_in_data = [col for col in item_tuple if col in value_data.columns]
    if not cols_in_data:
        print(f"【跳过】{group_title}，原因：无对应列")
        continue

    def match_group(row):
        non_null_items = row.dropna().index
        return (set(non_null_items) <= set(cols_in_data)) or (set(cols_in_data) <= set(non_null_items))

    matched_rows = value_data[cols_in_data].apply(match_group, axis=1)
    filtered_data = data_rem_dup.loc[matched_rows]

    if not filtered_data.empty:
        save_path = f"../data/{group_title}.pkl"
        filtered_data.to_pickle(save_path)
        print(f"已保存：{group_title} -> {save_path}")
    else:
        print(f"【无数据】{group_title}：没有满足条件的检测记录")


对每个特征检测组分别进行频次统计，结果绘制成图表：

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

pkl_folder = '../data_sub'

pkl_files = [f for f in os.listdir(pkl_folder) if f.endswith('.pkl')]

columns_to_analyze = data_rem_dup.columns[8:]
data_rem_dup['采集小时'] = pd.to_datetime(data_rem_dup['采集时间']).dt.hour


# 现有每日入院人数，对每张图还要给出每日检测数量和入院人数的比例的统计，以下是每日入院人数统计
# heatmap_data = pd.DataFrame(0.0, index=columns_to_analyze, columns=range(24))

# for col in columns_to_analyze:
#     if col in data_rem_dup.columns:
#         hour_counts = data_rem_dup.loc[data_rem_dup[col].notnull(), '采集小时'].value_counts()
#         total_counts = hour_counts.sum()
#         for hour, count in hour_counts.items():
#             heatmap_data.loc[col, hour] = count
#             heatmap_data.loc[col, hour] = heatmap_data.loc[col, hour] / total_counts



for file in pkl_files:
    file_path = os.path.join(pkl_folder, file)
    group_name = os.path.splitext(file)[0]
    
    if group_name not in group_map:
        print(f"文件 {file} 无对应 group_map 项，跳过。")
        continue

    print(f"正在处理: {file}")
    df = pd.read_pickle(file_path)
    df['采集时间'] = pd.to_datetime(df['采集时间'])
    df['采集小时'] = df['采集时间'].dt.hour
    df['采集日期'] = df['采集时间'].dt.to_period('D').dt.start_time
    df['采集年份'] = df['采集时间'].dt.year
    df['采集星期'] = df['采集时间'].dt.dayofweek  # 0周一，6周日

    group_columns = group_map[group_name]
    available_columns = [col for col in group_columns if col in df.columns]

    if not available_columns:
        print(f"  {group_name} 中无可用列，跳过。")
        continue

    # 按日统计
    daily_counts = []
    for col in available_columns:
        daily_count = df.loc[df[col].notnull()].groupby('采集日期').size()
        daily_counts.append(daily_count)
    total_daily_counts = sum(daily_counts)


    plt.figure(figsize=(14, 4))
    # total_daily_counts.sort_index().plot(kind='bar', width=1)
    plt.bar(daily_count.index, daily_count.values, width=2, label='入院人数', alpha=0.7)
    plt.title(f'{group_name} - 每日采集次数')
    plt.xlabel('日期')
    plt.ylabel('采集次数')
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

    # # 按周统计
    # weekly_counts = []
    # for col in available_columns:
    #     weekly_count = df.loc[df[col].notnull()].groupby(df['采集日期'].dt.to_period('W').apply(lambda r: r.start_time)).size()
    #     weekly_counts.append(weekly_count)
    # total_weekly_counts = sum(weekly_counts)

    # plt.figure(figsize=(14, 4))
    # # total_weekly_counts.sort_index().plot()
    # plt.bar(weekly_count.index, weekly_count.values, width=2, label='入院人数', alpha=0.7)
    # plt.title(f'{group_name} - 每周采集次数')
    # plt.xlabel('周起始日期')
    # plt.ylabel('采集次数')
    # plt.grid(axis='y', linestyle='--', alpha=0.5)
    # plt.tight_layout()
    # plt.show()

    # 按星期几统计
    weekday_counts = []
    for col in available_columns:
        weekday_count = df.loc[df[col].notnull()].groupby('采集星期').size()
        weekday_counts.append(weekday_count)
    total_weekday_counts = sum(weekday_counts)

    plt.figure(figsize=(8, 4))
    total_weekday_counts.sort_index().plot(kind='bar')
    plt.title(f'{group_name} - 每周采集分布')
    plt.xlabel('星期')
    plt.xticks(ticks=range(7), labels=['周一', '周二', '周三', '周四', '周五', '周六', '周日'])
    plt.ylabel('采集次数')
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

    # 按小时统计
    heatmap_data = pd.DataFrame(0.0, index=available_columns, columns=range(24))

    for col in available_columns:
        hour_counts = df.loc[df[col].notnull(), '采集小时'].value_counts()
        total_counts = hour_counts.sum()
        for hour, count in hour_counts.items():
            heatmap_data.loc[col, hour] = count / total_counts

    plt.figure(figsize=(10, 0.6 * len(available_columns)))
    sns.heatmap(
        heatmap_data, cmap='BuGn', linewidths=0.1,
        cbar_kws={'label': '采集占比', 'shrink': 0.5}
    )
    plt.title(f'{group_name} - 检查采集时间热力图（按小时）')
    plt.xlabel('小时')
    plt.ylabel('检查项目')
    plt.tight_layout()
    plt.show()


将特征检测组热力图与入院人数热力图进行对比，可总结出以下特殊特征：
1. 炎症因子检测组开始时间较晚，22年初从才开始有检测案例，且且出现多处无检测样例的时间段
2. 甲状腺功能评估在2021年3月到2022年2月期间没有检测样例
3. 肾功能及钙磷代谢评估在22年之前相对较少，23年3月疫情放开后样例显著增多
4. 骨形成标志物同样在22年之前相对较少，23年3月疫情放开后样例显著增多